In [1]:
from __future__ import division, absolute_import

import os
import sys
import random

import numpy as np
import datetime

#root
absPath = '/home/angela/padding_EBI/'
sys.path.insert(0, absPath)

#from src.Target import Target

np.random.seed(8)
random.seed(8)

from src.preprocessing import *
from src.model_architecture import *
from src.training_model import *
from src.postprocessing import *
from src.comparing_results import *
#from src.callbacks import *
%matplotlib inline

Using TensorFlow backend.


### Defining parameters

In [2]:
list_paddings = [
    'post_padding', 'pre_padding', 'mid_padding', 'strf_padding', 'ext_padding', 
                'rnd_padding', 
                 'aug_padding',
                "zoom_padding"
]

#hierarchy of folders: annotation/dataset/architecture/n_neurs/task/padding
folder = 'EC_number/archaea/rnn_conv/256rnn/'
column = "EC number"

n_folds = 10

dicti = creating_dict()

max_lenn = 1000

n_class = 7 #number of classes to output
drop_per = 0.2 #Input dropout 
n_neur = [313,76]
drop_hid = 0.5
dict_size = len(dicti)
final_act = "softmax"
n_filt = 64 #None
kernel_size = [5] #[1,3,5,9,15]#[5] #None
pool_size = 10 #None
n_hid = 256 #None

batch_size = 54
epochss = 500

callbacks_list = [None, None, None, None, False, False, False, False]
task = "task2/"
#architecture = "only_denses"
#architecture = "conv_dense"
architecture = "rnn_conv"
#architecture = "stack_conv"

In [3]:
lr=.0001
optimizer = Adam(lr=lr)

### Loading stuff

In [4]:
###LOADING ENZYME INDICES BECAUSE THIS IS TASK 2

file_idcs_aug = os.path.join(absPath, 'data/', folder, 'idcs_aug_enz.pickle')

with open(file_idcs_aug, "rb") as input_file:
    k_aug_indices = pickle.load(input_file)
    
file_idcs = os.path.join(absPath, 'data/', folder, 'idcs_data_enz.pickle')

with open(file_idcs, "rb") as input_file:
    splitting_sets = pickle.load(input_file)

### Deep learning model - task 2

In [5]:
generators_dict = {} 
for model_type in list_paddings:
    generators_dict[model_type] = trainval_generators(splitting_sets, k_aug_indices, model_type, 
                                                      folder, batch_size, 'labels_task2', True)
    
model = building_convrnn_model_task2(max_lenn, dict_size, n_neur, n_class, drop_per, drop_hid, 
                                 n_filt, kernel_size, pool_size, n_hid, final_act, folder, optimizer)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1000, 26)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1000, 26)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 1000, 64)          8384      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 100, 64)           0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 100, 64)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 512)               657408    
_________________________________________________________________
dense_1 (Dense)              (None, 313)               160569    
__________

In [6]:
# Ejemplo para saber la lista de layers del modelo cnn y del rnn
paddng = "mid_padding"
n_fold = "0"
cnn_model_path = "".join((absPath, "data/checkpoint/EC_number/archaea/1conv/64filts_size5/", 
                     task, paddng, "/", n_fold, "/"))
cnn_model_file = os.listdir(cnn_model_path)[0]
cnn_model = load_model("".join((cnn_model_path, cnn_model_file)))

cnn_model.layers

In [7]:
#warm_start(model, paddng, folder, task, n_fold, 2)

In [8]:
model.layers

### Defining functions

In [9]:
def warm_start(model_rnn, model_type, folder, task, idx, n_layer):
    cnn_model_path = "".join((absPath, "data/checkpoint/EC_number/archaea/1conv/64filts_size5/", 
                     task, model_type, "/", str(idx), "/"))
    cnn_model_file = os.listdir(cnn_model_path)[0]
    cnn_model = load_model("".join((cnn_model_path, cnn_model_file)))
    conv_layer_cnn = cnn_model.layers[n_layer].get_weights()
    model_rnn.layers[n_layer].set_weights(conv_layer_cnn)
    model_rnn.layers[n_layer].trainable = False
    return model_rnn

def model_training_simplified(model_type, folder, task, idx, callbacks_list, train_generator, val_generator, 
                              architecture, max_len, dict_size,
                             batch_size, n_neur, n_class, drop_per, drop_hid, final_act,
                             epochss, len_train, len_val, n_filt, kernel_size, pool_size, nhid, 
                             optimizer, n_layer):
    folder_cp = ''.join(string for string in [folder, task, model_type, '/', str(idx)])
    if not os.path.exists(os.path.join(absPath, 'data/checkpoint/', folder_cp)):
        os.makedirs(''.join(string for string in [absPath, 'data/checkpoint/', folder_cp]))
    callbacks_list = calling_callbacks(folder_cp, folder, model_type, callbacks_list[0], callbacks_list[1], 
                                        callbacks_list[2], callbacks_list[3], callbacks_list[4], 
                                        callbacks_list[5], callbacks_list[6], callbacks_list[7])
    folder_task =  ''.join(string for string in [folder, task])
    model_rnn = model_choice(architecture, task, folder, max_len, dict_size, n_neur, n_class, drop_per, drop_hid, 
                            final_act, n_filt=n_filt, kernel_size=kernel_size, pool_size=pool_size, 
                         nhid = nhid, optimizer=optimizer)
    model = warm_start(model_rnn, model_type, folder, task, idx, n_layer)
    #recompiling to fix the weights
    model.compile(loss='categorical_crossentropy', optimizer = optimizer, metrics=['accuracy'])
    #writing log file 
    log_file = ''.join(string for string in [absPath, 'data/checkpoint/', folder, task, 'log_file.txt' ]) 
    f = open(log_file, 'a+')
    print('Model type: %s \n' % model_type, file=f)
    print('Fold: %i \n' % idx, file=f)
    start = time.time()
    formatted_time = datetime.datetime.now()
    print('Starting time: %s \n' % formatted_time, file=f)
    history = model.fit_generator(generator=train_generator, 
                            validation_data=val_generator,
                            steps_per_epoch= int(len_train/batch_size),
                            validation_steps=int(len_val/batch_size),
                            epochs=epochss,
                            callbacks=callbacks_list,
                            verbose=1)
    end = time.time()
    formatted_endtime = datetime.datetime.now()
    print('Finishing time: %s \n' % formatted_endtime, file=f)
    count_time(start, end, folder, model_type, task)
    saving_results(history, model_type, folder, task, idx, True)
    f.close()    

### Training model

In [ ]:
for model_type in list_paddings:
    generators = generators_dict[model_type]
    for idx,i in enumerate(generators):
        if model_type == "aug_padding":
            i_train, i_val, i_test = k_aug_indices[idx]
        else: 
            i_train, i_val, i_test = splitting_sets[idx]
        len_train, len_val, len_test = len(i_train), len(i_val), len(i_test)
        train_generator, val_generator = i
        # a partir de aqui hay que añadir el tema del warm start
        model_training_simplified(model_type, folder, task, idx, callbacks_list, train_generator, 
                        val_generator, architecture, max_lenn, dict_size, batch_size, 
                        n_neur, n_class, drop_per, drop_hid, final_act, epochss, 
                        len_train, len_val,
                        n_filt = n_filt, kernel_size=kernel_size, pool_size=pool_size,
                      nhid=n_hid, optimizer=optimizer, n_layer=2)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 1000, 26)          0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 1000, 26)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 1000, 64)          8384      
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 100, 64)           0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 100, 64)           0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 512)               657408    
_________________________________________________________________
dense_4 (Dense)              (None, 313)               160569    
__________